In [7]:
#import packages
import sys
import glob, os, fnmatch

import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

# set source path
codepath = os.path.abspath("/Users/Pavel/Documents/repos/UNet/")
if codepath not in sys.path:
    sys.path.append(codepath)

datapath = os.path.abspath("/Users/Pavel/Documents/repos/UNet/docs/data/cifar/")

#automatically track changes in the source code
%load_ext autoreload
%autoreload 2

In [8]:
from UNet.data_handling.base import BaseDataLoader 
from UNet.models.unet import UNet
from UNet.training.base_trainer import BaseTrainer
from UNet.data_handling.unetdataset import UNetDataset
from UNet.classes.preprocess import Resize

In [9]:
SIZE_X = (572, 572) # size of input images
SIZE_Y = (388, 388) # size of input segmented images

LEARNING_RATE = 5e-3 # learning rate

In [10]:
unet_dataset = UNetDataset(
    root_dir="/Users/Pavel/Documents/repos/UNet/docs/data/PH2_Dataset_images/",
    images_folder="images",
    masks_folder="masks", 
    extension="*.bmp", 
    transform=transforms.Compose([Resize(SIZE_X, SIZE_Y)]))

In [11]:
unet_dataset.images_list

array(['IMD002 copy 19.bmp', 'IMD002 copy 31.bmp', 'IMD002 copy 25.bmp',
       'IMD002 copy 9.bmp', 'IMD002 copy 8.bmp', 'IMD002 copy 24.bmp',
       'IMD002 copy 30.bmp', 'IMD002 copy 18.bmp', 'IMD002 copy 26.bmp',
       'IMD002 copy 32.bmp', 'IMD002 copy 33.bmp', 'IMD002 copy 27.bmp',
       'IMD002 copy 23.bmp', 'IMD002 copy 37.bmp', 'IMD002 copy 36.bmp',
       'IMD002 copy 22.bmp', 'IMD002 copy 34.bmp', 'IMD002 copy 20.bmp',
       'IMD002 copy 21.bmp', 'IMD002 copy 35.bmp', 'IMD002 copy.bmp',
       'IMD002.bmp', 'IMD002 copy 38.bmp', 'IMD002 copy 10.bmp',
       'IMD002 copy 11.bmp', 'IMD002 copy 39.bmp', 'IMD002 copy 13.bmp',
       'IMD002 copy 3.bmp', 'IMD002 copy 2.bmp', 'IMD002 copy 12.bmp',
       'IMD002 copy 16.bmp', 'IMD002 copy 6.bmp', 'IMD002 copy 7.bmp',
       'IMD002 copy 17.bmp', 'IMD002 copy 15.bmp', 'IMD002 copy 29.bmp',
       'IMD002 copy 5.bmp', 'IMD002 copy 4.bmp', 'IMD002 copy 28.bmp',
       'IMD002 copy 14.bmp'], dtype='<U18')

In [12]:
# Create the corresponding dataloader for training and validation
data_loader = BaseDataLoader(dataset=unet_dataset,
                             batch_size=4,
                             validation_split=0.1
                            )

validation split =  0.1
int(len(dataset) - split) 36
split 4
preparing train and val loaders ... 


In [13]:



data, targets = next(iter(data_loader.train_loader))

# Inspect the data
print(data.shape)      # torch.Size([64, 1, 28, 28])
print(targets.shape)  

torch.Size([4, 572, 572, 3])
torch.Size([4, 388, 388])


In [14]:
# Define the model
unet_model = UNet()
save_dir = os.getcwd()+'/runs/exp1'

In [15]:
# Define the loss function and the optimizer
bce_loss = nn.BCEWithLogitsLoss()
unet_optimizer = optim.AdamW(unet_model.parameters(), lr=LEARNING_RATE)

In [16]:
# Initialize the trainer
trainer = BaseTrainer(model = unet_model,
                      criterion = bce_loss,
                      optimizer = unet_optimizer,
                      data_loader = data_loader,
                      epochs=100)

In [17]:
trainer.val_loader

In [18]:
# Start training
trainer.train()

* Epoch 1/100
training step ... 
avg train loss tensor(0.4891, grad_fn=<DivBackward0>)


TypeError: 'NoneType' object is not callable

In [5]:
# inheritance " in an old-fashioned and unflexible way  " - don't use it

class Female_Grandparent:
    def __init__(self):
        self.grandma_name = 'Grandma'

class Male_Grandparent:
    def __init__(self):
        self.grandpa_name = 'Grandpa'

class Parent(Female_Grandparent, Male_Grandparent):
    def __init__(self):
        Female_Grandparent.__init__(self)
        Male_Grandparent.__init__(self)

        self.parent_name = 'Parent Class'

class Child(Parent):
    def __init__(self):
        Parent.__init__(self)
        
        #---------------------------------------------------------------------------------------#
        for cls in Parent.__bases__: # This block grabs the classes of the child
             cls.__init__(self)      # class (which is named 'Parent' in this case), 
                                     # and iterates through them, initiating each one.
                                     # The result is that each parent, of each child,
                                     # is automatically handled upon initiation of the 
                                     # dependent class. WOOT WOOT! :D
        #---------------------------------------------------------------------------------------#


In [6]:
g = Female_Grandparent()
print (g.grandma_name)

p = Parent()
print (p.grandma_name)

child = Child()

print (child.grandma_name)

Grandma
Grandma
Grandma


In [1]:
# comparison of inheritance whilst using super and not using super

class SomeBaseClass(object):
    def __init__(self):
        print('SomeBaseClass.__init__(self) called')
    
class UnsuperChild(SomeBaseClass):
    def __init__(self):
        print('UnsuperChild.__init__(self) called')
        SomeBaseClass.__init__(self)
    
class SuperChild(SomeBaseClass):
    def __init__(self):
        print('SuperChild.__init__(self) called')
        super().__init__()


In [2]:
# add another class iheriting from the base class
class InjectMe(SomeBaseClass):
    def __init__(self):
        print('InjectMe.__init__(self) called')
        super().__init__()

        
# Now you want to have a class  inheriting from the newly created InjectMe and the UnsuperChild
# Both inherit from the Base class
# InjectMe uses super, but UnsuperChild hard-codes the inheritance from the BaseCLass
class UnsuperInjector(UnsuperChild, InjectMe): pass

# Now you want to have a class  inheriting from the newly created InjectMe and the SuperChild
# Both inherit from the Base class
# Both InjectMe and SuperChild use super to inherit from the BaseCLass
class SuperInjector(SuperChild, InjectMe): pass

In [3]:
# Using the un-super child fails to inject the dependency 
# because the child you're using has hard-coded the method to be called after its own:
o = UnsuperInjector()

UnsuperChild.__init__(self) called
SomeBaseClass.__init__(self) called


In [4]:
# However, the class with the child that uses super can correctly inject the dependency:
o2 = SuperInjector()

SuperChild.__init__(self) called
InjectMe.__init__(self) called
SomeBaseClass.__init__(self) called
